In [1]:
%matplotlib inline
import os
import glob
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pathlib
from matplotlib.lines import Line2D
from PIL import Image
import arff


In [2]:
_METRICS_FILE = './files/benchmark'
_METRIC = 'kappa'
_NUM_CLASSIFIER =  10

directory = './files/statistic_test/'+_METRIC+'/'

In [3]:
try:
    #read csv file
    metrics = pd.read_csv(_METRICS_FILE+'.csv')
except OSError as err:
    # if there is no csv file we read the excel file
    excel_file = pd.read_excel(_METRICS_FILE+'.xlsx')
    # transform the excel file in a csv file
    excel_file.to_csv(_METRICS_FILE+'.csv', index=False)
    # read the new csv file 
    metrics = pd.read_csv(_METRICS_FILE+'.csv')
except:
    print("ERROR WHILE READING METRICS FILE")

metrics = metrics[metrics.instances>1000]

metrics.sample(5)

,dataset,classifier,instances,attributes,classes,accuracy,kappa,train_time_mean,test_time_mean,total_time_mean,train_time,test_time,total_time
2326,bankmarketingnorm,MOA_HoeffdingTree,45211,16,2,0.575196,0.126896,0.000025,0.000005,0.000029,0.112023,0.021313,0.133336
1866,Symbols,MOA_OCBoost,1020,398,6,0.167850,0.001200,0.033927,0.015161,0.049088,3.460557,1.546385,5.006942
2017,TwoLeadECG,MOA_Perceptron,1162,82,2,0.000000,0.000000,0.000002,0.000012,0.000014,0.000206,0.001425,0.001631
662,FacesUCR,sklearn_BernoulliNB,2240,131,14,0.704066,0.659858,0.000373,0.000207,0.000580,0.083444,0.046413,0.129857
235,CinCECGtorso,MOA_DecisionStump,1420,1639,4,0.286042,0.084846,0.000558,0.000001,0.000560,0.079299,0.000188,0.079487


In [4]:
metrics = metrics.pivot(index='dataset', columns='classifier', values=[_METRIC])
metrics.sample(10)

kappa                       \
classifier           MOA_ASHoeffdingTree MOA_ActiveClassifier   
dataset                                                         
InsectWingbeatSound             0.587159             0.524469   
spambase                        0.923170             0.113965   
ECG5000                         0.750474             0.744013   
covtype                         0.594820             0.927403   
pendigits                       0.849582             0.816879   
Symbols                         0.888874             0.761276   
pokerhand                       0.324964             0.017863   
ShapesAll                       0.565363             0.045274   
UWaveGestureLibraryZ            0.503895             0.481675   
Wafer                           0.356147             0.182273   

                                                                   \
classifier           MOA_AdaHoeffdingOptionTree MOA_DecisionStump   
dataset                                                             
InsectWingbeatSound                    0.587159          0.081000   
spambase                               0.923170          0.003472   
ECG5000                                0.750474          0.734588   
covtype                                0.904718          0.000003   
pendigits                              0.848946          0.095415   
Symbols                                0.888874          0.004393   
pokerhand                              0.302924          0.016286   
ShapesAll                              0.555479          0.002315   
UWaveGestureLibraryZ                   0.503895          0.122621   
Wafer                                  0.337769          0.321954   

                                                                        \
classifier           MOA_HoeffdingAdaptiveTree MOA_HoeffdingOptionTree   
dataset                                                                  
InsectWingbeatSound                   0.588115                0.587159   
spambase                              0.991387                0.919857   
ECG5000                               0.754102                0.750474   
covtype                               0.920922                0.834623   
pendigits                             0.853065                0.845548   
Symbols                               0.887090                0.888874   
pokerhand                             0.171288                0.333284   
ShapesAll                             0.546244                0.565363   
UWaveGestureLibraryZ                  0.506894                0.503895   
Wafer                                 0.375472                0.341560   

                                                                            \
classifier           MOA_HoeffdingTree MOA_LeveragingBag MOA_MajorityClass   
dataset                                                                      
InsectWingbeatSound           0.587159          0.589451      0.000000e+00   
spambase                      0.923170          0.999698      8.626447e-01   
ECG5000                       0.750474          0.751867      1.620135e-03   
covtype                       0.594820          0.956677      3.166315e-06   
pendigits                     0.849582          0.867245      0.000000e+00   
Symbols                       0.888874          0.887084      0.000000e+00   
pokerhand                     0.324964          0.929886      3.727942e-07   
ShapesAll                     0.565363          0.499089      0.000000e+00   
UWaveGestureLibraryZ          0.503895          0.507329      0.000000e+00   
Wafer                         0.356147          0.963474      2.784950e-01   

                                             ...                        \
classifier           MOA_NaiveBayes          ...          MOA_SPegasos   
dataset                                      ...                         
InsectWingbeatSound        0.587229          ...              0.000000   
spambase         

In [5]:
metrics = metrics.drop((_METRIC, 'keras_3_Dilated_Conv'),1)
metrics = metrics.dropna()

In [6]:
metrics = metrics.reindex((-metrics.mean()).sort_values().index, axis=1)
metrics = metrics[[metrics.columns[i] for i in range(_NUM_CLASSIFIER)]]
metrics.head()

kappa                        \
classifier            keras_parallel_3_Dilated_Conv sklearn_MLPClassifier   
dataset                                                                     
ChlorineConcentration                      0.283538              0.081596   
CinCECGtorso                               0.979644              0.981750   
ECG5000                                    0.901193              0.877330   
ElectricDevices                            0.681580              0.526164   
FaceAll                                    0.934187              0.728645   

                                                                 \
classifier            MOA_LeveragingBag MOA_HoeffdingOptionTree   
dataset                                                           
ChlorineConcentration          0.062581                0.000711   
CinCECGtorso                   0.225028                0.233757   
ECG5000                        0.751867                0.750474   
ElectricDevices                0.468425                0.455576   
FaceAll                        0.849261                0.735665   

                                                                     \
classifier            MOA_SingleClassifierDrift MOA_ASHoeffdingTree   
dataset                                                               
ChlorineConcentration                  0.122219            0.000711   
CinCECGtorso                           0.237493            0.233757   
ECG5000                                0.771611            0.750474   
ElectricDevices                        0.455576            0.455576   
FaceAll                                0.829490            0.735665   

                                                                        \
classifier            MOA_HoeffdingTree MOA_OzaBoost MOA_OzaBoostAdwin   
dataset                                                                  
ChlorineConcentration          0.000711     0.000699          0.000699   
CinCECGtorso                   0.233757     0.000000          0.000000   
ECG5000                        0.750474     0.823303          0.842649   
ElectricDevices                0.455576     0.462254          0.457064   
FaceAll                        0.735665     0.871212          0.868891   

                                                 
classifier            MOA_HoeffdingAdaptiveTree  
dataset                                          
ChlorineConcentration                  0.000699  
CinCECGtorso                           0.233682  
ECG5000                                0.754102  
ElectricDevices                        0.451940  
FaceAll                                0.791064

In [7]:
file_path = directory+_METRIC+'_table.csv'
if not os.path.exists(directory):
    os.makedirs(directory)
metrics.to_csv(file_path, sep=',', encoding='utf-8')

In [8]:
with open(file_path, "r+") as f:
    lines = f.readlines()
    del lines[2]
    del lines[0]
    lines[0] = lines[0].replace('classifier',' ', 1 )
    f.seek(0)
    f.truncate()
    f.writelines(lines)

In [9]:
[c for c in metrics.columns]

[('kappa', 'keras_parallel_3_Dilated_Conv'),
 ('kappa', 'sklearn_MLPClassifier'),
 ('kappa', 'MOA_LeveragingBag'),
 ('kappa', 'MOA_HoeffdingOptionTree'),
 ('kappa', 'MOA_SingleClassifierDrift'),
 ('kappa', 'MOA_ASHoeffdingTree'),
 ('kappa', 'MOA_HoeffdingTree'),
 ('kappa', 'MOA_OzaBoost'),
 ('kappa', 'MOA_OzaBoostAdwin'),
 ('kappa', 'MOA_HoeffdingAdaptiveTree')]